In [ ]:
import pickle
from pathlib import Path

from tqdm.notebook import tqdm

import numpy as np

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
ls *pickle

In [ ]:
with open('specphot_phoenix.pickle', 'rb') as f:
    dct = pickle.load(f)
dct.keys()

In [ ]:
for k, v in dct.items():
    globals()[k] = v

In [ ]:
absmags.keys()0

In [ ]:
featuresl = features.copy()
featuresl['logteff'] = np.log10(featuresl.pop('teff'))
featuresl

In [ ]:
next(iter(featuresl.values())).shape, next(iter(absmags.values())).shape

In [ ]:
test_absmags = {k:v for k,v in absmags.items() if ('F814W' in k or 'F606W' in k)}
test_absmags.keys()

# SVMs? 

In [ ]:
from sklearn import svm

In [ ]:
X = np.array(list(test_absmags.values()))
y = featuresl['logg']

plt.scatter(X[0], X[1], c=y, s= 2, alpha=.5)
plt.colorbar()

In [ ]:
regr = svm.SVR()
regr.fit(X.T, y)

In [ ]:
plt.scatter(X[0], X[1], c=regr.predict(X.T), s= 2, alpha=.5)

Hmm.  Simple enough it might work?

In [ ]:
svms = {}

# this is the default, and it doesn't seem to do much better even trying the others...
kernel = 'rbf' # 'linear', 'poly', 'rbf', 'sigmoid'
degree = 3

for fnm, y in featuresl.items():
    svms[fnm] = regr = svm.SVR(kernel=kernel, degree=degree)
    regr.fit(X.T, y)

In [ ]:
fig, axs = plt.subplots(len(svms), 3, figsize=(15, 15))

for (ax1, ax2, ax3), (fnm, regr) in zip(axs, svms.items()):
    y = featuresl[fnm]
    yp = regr.predict(X.T)

    mi = np.min(np.concatenate([y, yp]))
    mx = np.max(np.concatenate([y, yp]))

    sc1 = ax1.scatter(X[0], X[1], c=y, s=2, alpha=.5, vmin=mi, vmax=mx)
    sc2 = ax2.scatter(X[0], X[1], c=yp, s=2, alpha=.5, vmin=mi, vmax=mx)
    plt.colorbar(sc1).set_label(fnm)

    sc3 = ax3.scatter(X[0], y-yp, c=X[1], s=2, alpha=.5)
    plt.colorbar(sc3).set_label('mag2')

With any choice of kernel we still have this (unsurprising) problem that the Fe/H dependence is hard to capture.

## Gaussian Process?

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import *
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

X = np.array(list(test_absmags.values()))
y = featuresl['logg']


kernel = 1.0 * ExpSineSquared(1.0, 5.0, periodicity_bounds=(1e-2, 1e1))
kernel = 1 * RBF(length_scale=.1, length_scale_bounds=(1e-2, 1e2))
kernel = 1 * Matern(length_scale=.1, nu=1.5)

gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

pl = make_pipeline(StandardScaler(), gaussian_process)
pl = gaussian_process

sub = np.random.permutation(y.size)[:100]
pl.fit(X.T[sub], y[sub])

In [ ]:
yp = pl.predict(X.T)

mi = np.min(np.concatenate([y, yp]))
mx = np.max(np.concatenate([y, yp]))

plt.scatter(X[0], X[1], c=y, s=2, alpha=.5, vmin=mi, vmax=mx)
plt.figure()
plt.scatter(X[0], X[1], c=yp, s=2, alpha=.5, vmin=mi, vmax=mx)

Hmm, it works but doesn't converge sometimes?

## Decision trees

In [ ]:
from sklearn import tree


X = np.array(list(test_absmags.values()))
y = featuresl['logg']


In [ ]:
dtree = tree.DecisionTreeRegressor(max_depth=None)


training_sub = np.random.permutation(y.size)[:100000]

dtree.fit(X.T[training_sub], y[training_sub])

In [ ]:
yp = dtree.predict(X.T)

mi = np.min(np.concatenate([y, yp]))
mx = np.max(np.concatenate([y, yp]))

plt.scatter(X[0], X[1], c=y, s=2, alpha=.5, vmin=mi, vmax=mx)
plt.figure()
plt.scatter(X[0], X[1], c=yp, s=2, alpha=.5, vmin=mi, vmax=mx)

multioutput?

In [ ]:
yall = np.array(list(featuresl.values()))

In [ ]:
mtree = tree.DecisionTreeRegressor(max_depth=None)

training_sub = np.random.permutation(y.size)[:500]

mtree.fit(X.T[training_sub], yall.T[training_sub])

In [ ]:
yp = mtree.predict(X.T)

fig, axs = plt.subplots(len(svms), 3, figsize=(15, 15))

for (ax1, ax2, ax3), (fnm, f), fyp in zip(axs, featuresl.items(), yp.T):
    y = featuresl[fnm]

    mi = np.min(np.concatenate([y, fyp]))
    mx = np.max(np.concatenate([y, fyp]))

    sc1 = ax1.scatter(X[0], X[1], c=y, s=2, alpha=.5, vmin=mi, vmax=mx)
    sc2 = ax2.scatter(X[0], X[1], c=fyp, s=2, alpha=.5, vmin=mi, vmax=mx)
    plt.colorbar(sc1).set_label(fnm)

    sc3 = ax3.scatter(X[0], y-fyp, c=X[1], s=2, alpha=.5)
    plt.colorbar(sc3).set_label('mag2')

In [ ]:
from sklearn import ensemble

In [ ]:
forest = ensemble.RandomForestRegressor()

training_sub = np.random.permutation(y.size)[:250000]

forest.fit(X.T[training_sub], yall.T[training_sub])

In [ ]:
yp = forest.predict(X.T)

fig, axs = plt.subplots(len(svms), 3, figsize=(15, 15))

for (ax1, ax2, ax3), (fnm, f), fyp in zip(axs, featuresl.items(), yp.T):
    y = featuresl[fnm]

    mi = np.min(np.concatenate([y, fyp]))
    mx = np.max(np.concatenate([y, fyp]))

    sc1 = ax1.scatter(X[0], X[1], c=y, s=2, alpha=.5, vmin=mi, vmax=mx)
    sc2 = ax2.scatter(X[0], X[1], c=fyp, s=2, alpha=.5, vmin=mi, vmax=mx)
    plt.colorbar(sc1).set_label(fnm)

    sc3 = ax3.scatter(X[0], y-fyp, c=X[1], s=2, alpha=.5)
    plt.colorbar(sc3).set_label('mag2')

Kinda works but still really fails to handle metalicity well.